In [46]:
import asana
from asana.rest import ApiException
from pprint import pprint
from dotenv import load_dotenv
import os
import requests
from helpers import get_gid_from_json
import pandas as pd
from datetime import datetime



In [47]:

gid_file_path = 'workspace.json'

load_dotenv()
access_token = os.getenv('ASANA_TOKEN')
workspace_gid = get_gid_from_json(gid_file_path)




configuration = asana.Configuration()
configuration.access_token = access_token
api_client = asana.ApiClient(configuration)


# create an instance of the API class
workspaces_api_instance = asana.WorkspacesApi(api_client)
opts_workspaces = {
    'opt_fields': "https://app.asana.com/api/1.0/workspaces" # list[str] | This endpoint returns a compact resource, which excludes some properties by default. To include those optional properties, set this query parameter to a comma-separated list of the properties you wish to include.
}


def get_workspace_gid(opts_workspaces):
    try:
        # Get multiple workspaces
        api_response = workspaces_api_instance.get_workspaces(opts_workspaces)
        for data in api_response:
            workspace_gid = int(data['gid'])
            print(f'Workspace GID: {workspace_gid}')
            return workspace_gid
    except ApiException as e:
        print("Exception when calling WorkspacesApi->get_workspaces: %s\n" % e)
        
        
workspace_gid = get_workspace_gid(opts_workspaces)



# create an instance of the API class
projects_api_instance = asana.ProjectsApi(api_client)
opts_project = {
    'workspace': workspace_gid # str | The workspace or organization to filter projects on.
}


def get_project_gid(opts_project):
    try:
        # Get multiple projects
        api_response = projects_api_instance.get_projects(opts_project)
        for data in api_response:
            project_name = data['name']
            project_gid = data['gid']
            
            project_details = {
                'project_name': project_name,
                'project_gid': project_gid
            }
            
            print(f'Project Name: {project_name}')
            print(f'Project GID: {project_gid}')

            return project_details
            
    except ApiException as e:
        print("Exception when calling ProjectsApi->get_projects: %s\n" % e)
        
        
project_details = get_project_gid(opts_project)






# create an instance of the API class
tasks_api_instance = asana.TasksApi(api_client)
project_gid = project_details['project_gid'] # str | Globally unique identifier for the project.
    
    
opts_tasks = {
    'opt_fields': "notes,created_at,memberships.section.name,assignee.name,tags.name,name,created_at,f'https://app.asana.com/api/1.0/projects/{project_gid}/tasks'", # list[str] | This endpoint returns a compact resource, which excludes some properties by default. To include those optional properties, set this query parameter to a comma-separated list of the properties you wish to include.
#     'opt_fields': "assignee.name,section,tags.name,name,f'https://app.asana.com/api/1.0/projects/{project_gid}/tasks'", # list[str] | This endpoint returns a compact resource, which excludes some properties by default. To include those optional properties, set this query parameter to a comma-separated list of the properties you wish to include.

}


def get_all_task(project_gid, opts_tasks):
    try:
        # Get tasks from a project
        api_response = tasks_api_instance.get_tasks_for_project(project_gid, opts_tasks)
        data_list = []
        for data in api_response:
            data_list.append(data)
        
        df = pd.DataFrame(data_list)
        
        return df
    
    except ApiException as e:
        print("Exception when calling TasksApi->get_tasks_for_project: %s\n" % e)
        
df = get_all_task(project_gid, opts_tasks)



Workspace GID: 1207998439135084
Project Name: AMM & MPLAN Enrollment
Project GID: 1207998440065110


In [48]:
def rename_column_names(df):
    column_mapping = {
        'created_at': 'INIT DATE',
        'memberships': 'STATUS',
        'assignee': 'PIC',
        'name': 'DESCRIPTION',
        'notes': 'PART / MPLAN NO.'
    }
    df = df.rename(columns=column_mapping)
    return df

df = rename_column_names(df)

# Function to extract the name from the email or handle None/empty cases
def extract_assignee_name(assignee_dict):
    if assignee_dict is None:
        return "<unassigned>"
    else:
        email = assignee_dict['name']
        first_name, last_name = email.split('@')[0].split('.')
        return f"{first_name.capitalize()} {last_name.capitalize()}"

# Apply the function to the 'assignee' column
df['PIC'] = df['PIC'].apply(lambda x: extract_assignee_name(x) if x else "<unassigned>")

def extract_date(timestamp):
    if pd.isna(timestamp) or timestamp == '':
        return ''
    return timestamp.split('T')[0]

# Apply the function to the 'created_at' column
df['INIT DATE'] = df['INIT DATE'].apply(lambda x: extract_date(x))

def extract_membership_name(memberships):
    try:
        if memberships and isinstance(memberships, list):
            if len(memberships) > 0 and isinstance(memberships[0], dict):
                return memberships[0].get('section', {}).get('name', '')
        return ''
    except Exception as e:
        print(f"Error processing memberships: {e}")
        return ''

# Apply the function to the 'memberships' column
df['STATUS'] = df['STATUS'].apply(lambda x: extract_membership_name(x))

def extract_names(tag_list):
    if not tag_list:
        return []
    return [tag['name'] for tag in tag_list]

df['tags'] = df['tags'].apply(lambda x: extract_names(x))

def determine_project_type(tags):
    if 'OLD' in tags:
        return 'OLD'
    elif 'NEW' in tags:
        return 'NEW'
    else:
        return '<no_project_type>'

df['PROJECT_TYPE'] = df['tags'].apply(lambda x: determine_project_type(x))

def determine_area_type(tags):
    if 'COFFEE' in tags:
        return 'COFFEE'
    elif 'MILK' in tags:
        return 'MILK'
    else:
        return '<no_area>'

df['AREA'] = df['tags'].apply(lambda x: determine_area_type(x))

def determine_task_type(tags):
    if 'TARGET' in tags:
        return 'TARGET'
    elif 'ADDITIONAL' in tags:
        return 'ADDITIONAL'
    else:
        return '<no_type>'

df['TYPE'] = df['tags'].apply(lambda x: determine_task_type(x))

def determine_doc_type(tags):
    if 'MPLAN' in tags:
        return 'MPLAN'
    elif 'AMM' in tags:
        return 'AMM'
    else:
        return '<no_doc_type>'

df['DOC_TYPE'] = df['tags'].apply(lambda x: determine_doc_type(x))



def get_section_names(df):
    column_names = df.columns.tolist()
    return column_names


project_list = get_section_names(df)

# project_list = [
#     "AUTOCOMPACTOR",
#     "HYDRAULIC PROJECT",
#     "GCU+",
#     "WATER HEATER",
#     "GCCD Transport Phase 1",
#     "SIR",
#     "IPTA VACUUM PUMP",
#     "ICIP",
#     "GC Van Unloading Facility",
#     "PEC 2020: PE Panel Cooling",
#     "PEC 2020: Process adaptation",
#     "FFE-RARE",
#     "PEC 2020: Spot Cooling",
#     "<X> VACUUM PUMP"
# ]

def find_project(tags):
    for project in project_list:
        if project in tags:
            return project
    return None

df['PROJECT'] = df['tags'].apply(lambda x: find_project(x))

def calculate_age(init_date_str):
    init_date = pd.to_datetime(init_date_str)
    today = datetime.now()
    age_days = (today - init_date).days
    return age_days

df['INIT_AGE (DAYS)'] = df['INIT DATE'].apply(lambda x: calculate_age(x))

def task_stories(row):
    return "Temporary REMARKS"

df['REMARKS'] = df.apply(lambda row: task_stories(row), axis=1)



df

# #TO EXCEL
# def save_to_excel(df: pd.DataFrame, filename: str = "PROCESSED_DATA.xlsx"):
#     """
#     Save the given DataFrame to an Excel file. Overwrite the file if it already exists.

#     Parameters:
#     df (pd.DataFrame): The DataFrame to save.
#     filename (str): The name of the Excel file to save. Default is "PROCESSED_DATA.xlsx".
#     """
#     # Save the DataFrame to an Excel file, overriding if it exists
#     df.to_excel(filename, index=False)

# # Example usage
# save_to_excel(df)

,gid,PIC,INIT DATE,STATUS,DESCRIPTION,PART / MPLAN NO.,tags,PROJECT_TYPE,AREA,TYPE,DOC_TYPE,PROJECT,INIT_AGE (DAYS),REMARKS
0,1208034272468521,Franciscarlo Tadena,2024-08-12,DONE,DOOR FLAP PNUEMATIC CYLINDER - REPAIR,,"[AUTOCOMPACTOR, COFFEE, MPLAN, OLD, TARGET]",OLD,COFFEE,TARGET,MPLAN,AUTOCOMPACTOR,2,Temporary REMARKS
1,1208034281348801,Franciscarlo Tadena,2024-08-12,DONE,14W ENERGY CHAIN - INSPECTION,,"[AUTOCOMPACTOR, COFFEE, MPLAN, OLD, TARGET]",OLD,COFFEE,TARGET,MPLAN,AUTOCOMPACTOR,2,Temporary REMARKS
2,1208034281348813,Franciscarlo Tadena,2024-08-12,DONE,6M DRIVE GEAR MOTOR - PM INSPECTION,,"[AUTOCOMPACTOR, COFFEE, MPLAN, OLD, TARGET]",OLD,COFFEE,TARGET,MPLAN,AUTOCOMPACTOR,2,Temporary REMARKS
3,1208034358879963,Franciscarlo Tadena,2024-08-12,DONE,3Y DRIVE GEAR MOTOR PM CHANGE GEAR OIL,,"[AUTOCOMPACTOR, COFFEE, MPLAN, OLD, TARGET]",OLD,COFFEE,TARGET,MPLAN,AUTOCOMPACTOR,2,Temporary REMARKS
4,1208034354473967,Franciscarlo Tadena,2024-08-12,DONE,3W DRIVE GEAR MOTOR - INSPECTION -,,"[AUTOCOMPACTOR, COFFEE, MPLAN, OLD, TARGET]",OLD,COFFEE,TARGET,MPLAN,AUTOCOMPACTOR,2,Temporary REMARKS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,1208050280492498,Franciscarlo Tadena,2024-08-14,ON-GOING FIRST DRAFT,Slide Bearing PTD II 125 Diverter valve\n,,"[AMM, COFFEE, OLD, TARGET, GCCD Transport Phas...",OLD,COFFEE,TARGET,AMM,GCCD Transport Phase 1,0,Temporary REMARKS
255,1208050280492496,Franciscarlo Tadena,2024-08-14,ON-GOING FIRST DRAFT,Plain Bearing PTD II 125 Diverter valve\n,,"[AMM, COFFEE, OLD, TARGET, GCCD Transport Phas...",OLD,COFFEE,TARGET,AMM,GCCD Transport Phase 1,0,Temporary REMARKS
256,1208050280492494,Franciscarlo Tadena,2024-08-14,ON-GOING FIRST DRAFT,Lip Seal PTD II 125 Diverter valve\n,,"[AMM, COFFEE, OLD, TARGET, GCCD Transport Phas...",OLD,COFFEE,TARGET,AMM,GCCD Transport Phase 1,0,Temporary REMARKS
257,1208050280492488,Franciscarlo Tadena,2024-08-14,ON-GOING FIRST DRAFT,Seal PTD II 125 Diverter valve\n,,"[AMM, COFFEE, OLD, TARGET, GCCD Transport Phas...",OLD,COFFEE,TARGET,AMM,GCCD Transport Phase 1,0,Temporary REMARKS


In [50]:

# gid_file_path = 'workspace.json'

# load_dotenv()
# access_token = os.getenv('ASANA_TOKEN')
# workspace_gid = get_gid_from_json(gid_file_path)


# configuration = asana.Configuration()
# configuration.access_token = access_token
# api_client = asana.ApiClient(configuration)


# create an instance of the API class
stories_api_instance = asana.StoriesApi(api_client)
task_gid = "1208050280492494" # str | The task to operate on.
opts = {
    'opt_fields': "assignee,assignee.name,created_at,created_by,created_by.name,custom_field,custom_field.date_value,custom_field.date_value.date,custom_field.date_value.date_time,custom_field.display_value,custom_field.enabled,custom_field.enum_options,custom_field.enum_options.color,custom_field.enum_options.enabled,custom_field.enum_options.name,custom_field.enum_value,custom_field.enum_value.color,custom_field.enum_value.enabled,custom_field.enum_value.name,custom_field.id_prefix,custom_field.is_formula_field,custom_field.multi_enum_values,custom_field.multi_enum_values.color,custom_field.multi_enum_values.enabled,custom_field.multi_enum_values.name,custom_field.name,custom_field.number_value,custom_field.representation_type,custom_field.resource_subtype,custom_field.text_value,custom_field.type,dependency,dependency.created_by,dependency.name,dependency.resource_subtype,duplicate_of,duplicate_of.created_by,duplicate_of.name,duplicate_of.resource_subtype,duplicated_from,duplicated_from.created_by,duplicated_from.name,duplicated_from.resource_subtype,follower,follower.name,hearted,hearts,hearts.user,hearts.user.name,html_text,is_editable,is_edited,is_pinned,liked,likes,likes.user,likes.user.name,new_approval_status,new_date_value,new_dates,new_dates.due_at,new_dates.due_on,new_dates.start_on,new_enum_value,new_enum_value.color,new_enum_value.enabled,new_enum_value.name,new_multi_enum_values,new_multi_enum_values.color,new_multi_enum_values.enabled,new_multi_enum_values.name,new_name,new_number_value,new_people_value,new_people_value.name,new_resource_subtype,new_section,new_section.name,new_text_value,num_hearts,num_likes,offset,old_approval_status,old_date_value,old_dates,old_dates.due_at,old_dates.due_on,old_dates.start_on,old_enum_value,old_enum_value.color,old_enum_value.enabled,old_enum_value.name,old_multi_enum_values,old_multi_enum_values.color,old_multi_enum_values.enabled,old_multi_enum_values.name,old_name,old_number_value,old_people_value,old_people_value.name,old_resource_subtype,old_section,old_section.name,old_text_value,path,previews,previews.fallback,previews.footer,previews.header,previews.header_link,previews.html_text,previews.text,previews.title,previews.title_link,project,project.name,resource_subtype,source,sticker_name,story,story.created_at,story.created_by,story.created_by.name,story.resource_subtype,story.text,tag,tag.name,target,target.created_by,target.name,target.resource_subtype,task,task.created_by,task.name,task.resource_subtype,text,type,uri", # list[str] | This endpoint returns a compact resource, which excludes some properties by default. To include those optional properties, set this query parameter to a comma-separated list of the properties you wish to include.
}

try:
    # Get stories from a task
    api_response = stories_api_instance.get_stories_for_task(task_gid, opts)
    for data in api_response:
        pprint(data)
except ApiException as e:
    print("Exception when calling StoriesApi->get_stories_for_task: %s\n" % e)


{'created_at': '2024-08-14T05:17:06.772Z',
 'created_by': {'gid': '1207998336820177', 'name': 'Francis Carlo A. Tadena'},
 'gid': '1208051528023518',
 'html_text': '<b><a '
              'href="https://app.asana.com/0/profile/1207998439135094">Francis '
              'Carlo A. Tadena</a></b> added to <a '
              'href="https://app.asana.com/0/1207998440065110/1207998440065110">AMM '
              '&amp; MPLAN Enrollment</a>',
 'is_editable': False,
 'new_name': None,
 'previews': [],
 'project': {'gid': '1207998440065110', 'name': 'AMM & MPLAN Enrollment'},
 'resource_subtype': 'added_to_project',
 'source': 'web',
 'target': {'created_by': {'gid': '1207998336820177', 'resource_type': 'user'},
            'gid': '1208050280492494',
            'name': 'Lip Seal PTD II 125 Diverter valve\n',
            'resource_subtype': 'default_task'},
 'text': 'Francis Carlo A. Tadena added to AMM & MPLAN Enrollment',
 'type': 'system'}
{'assignee': {'gid': '1207901010306940',
              